<a href="https://colab.research.google.com/github/wangbxj1234/offset_pt_try/blob/main/uniform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch

In [ ]:
a=np.random.rand(32,2048,3)#########set from numpy
a.shape

(32, 2048, 3)

In [ ]:
points = torch.Tensor(a)#######to torch
points.shape

torch.Size([32, 2048, 3])

In [ ]:
def farthest_point_sample(xyz, npoint):
    """
    Input:
        xyz: pointcloud data, [B, N, 3]
        npoint: number of samples
    Return:
        centroids: sampled pointcloud index, [B, npoint]
    """
    device = xyz.device
    B, N, C = xyz.shape
    centroids = torch.zeros(B, npoint, dtype=torch.long).to(device)
    distance = torch.ones(B, N).to(device) * 1e10
    farthest = torch.randint(0, N, (B,), dtype=torch.long).to(device)
    batch_indices = torch.arange(B, dtype=torch.long).to(device)
    for i in range(npoint):
        centroids[:, i] = farthest
        centroid = xyz[batch_indices, farthest, :].view(B, 1, 3)
        dist = torch.sum((xyz - centroid) ** 2, -1)
        distance = torch.min(distance, dist)
        farthest = torch.max(distance, -1)[1]
    return centroids

def index_points(points, idx):
    """
    Input:
        points: input points data, [B, N, C]
        idx: sample index data, [B, S, [K]]
    Return:
        new_points:, indexed points data, [B, S, [K], C]
    """
    raw_size = idx.size()
    idx = idx.reshape(raw_size[0], -1)
    res = torch.gather(points, 1, idx[..., None].expand(-1, -1, points.size(-1)))
    return res.reshape(*raw_size, -1)


In [ ]:
fps_idx = farthest_point_sample(points, 1024)##############按照fps提取1维index
torch.cuda.empty_cache()#####清除显存
pointcloud = index_points(points, fps_idx)########### 按照index 返回 3维点。
pointcloud.shape

torch.Size([32, 1024, 3])

In [ ]:
pointcloud=pointcloud.data.numpy()################## TO NUMPY
pointcloud.shape

(32, 1024, 3)